Importing Dependencies

In [104]:
import pandas as pd

Loading health and population data

In [105]:
Health_file = pd.read_csv("HEALTH_STAT.csv")
pop_file = pd.read_csv("HISTPOP.csv")

health_df = pd.DataFrame(Health_file)
pop_df = pd.DataFrame(pop_file)

In [106]:
pop_df.head()

,LOCATION,Country,SEX,Sex,AGE,Age,TIME,Time,Value,Flag Codes,Flags
0,AUS,Australia,W,Women,TOTAL,Total,1964,1964,5516400.0,NaN,NaN
1,AUS,Australia,W,Women,TOTAL,Total,1965,1965,5626400.0,NaN,NaN
2,AUS,Australia,W,Women,TOTAL,Total,1966,1966,5757910.0,NaN,NaN
3,AUS,Australia,W,Women,TOTAL,Total,1967,1967,5859757.0,NaN,NaN
4,AUS,Australia,W,Women,TOTAL,Total,1968,1968,5965400.0,NaN,NaN


Cleaning Health Data

In [107]:
health_df["Variable"].values

array(['Females at birth', 'Females at birth', 'Females at birth', ...,
       'Males at age 80', 'Males at age 80', 'Males at age 80'],
      dtype=object)

In [108]:
pop_df_total = pop_df[pop_df["SEX"] == "T"]
pop_df_clean = pop_df_total.drop(["SEX", "Sex", "AGE", "Age", "TIME", "Flag Codes", "Flags"], axis=1)
population_df = pop_df_clean.rename(columns={"Time":"Year", "Value": "Population"})
population_df.head()

,LOCATION,Country,Year,Population
110,AUS,Australia,1964,11121600.0
111,AUS,Australia,1965,11340900.0
112,AUS,Australia,1966,11599498.0
113,AUS,Australia,1967,11799078.0
114,AUS,Australia,1968,12008635.0


In [109]:
health_df_total = health_df[health_df["Variable"] == "Total population at birth"]
health_df_total_cl = health_df_total.drop(["Variable", "VAR", "UNIT", "Measure", "YEA", "Flag Codes", "Flags"], axis=1)
health_stats = health_df_total_cl.rename(columns={"Value":"Life Expectancy at Birth"})
health_stats.head()

,COU,Country,Year,Life Expectancy at Birth
560,AUS,Australia,1964,70.7
561,AUS,Australia,1965,71.0
562,AUS,Australia,1966,70.8
563,AUS,Australia,1967,71.2
564,AUS,Australia,1968,70.9


Merging population and health stats to keep all of the population info 

In [110]:
pop_life = pd.merge(health_stats, population_df, how="right", on=["Country", "Year"])

In [111]:
pop_life = pop_life.drop("LOCATION", axis=1)

In [112]:
pop_life_cl = pop_life.drop_duplicates(subset=["Year","Country"])


In [114]:
pop_life_cl.shape[0]

3080

Checking to see how many missing values there are

In [115]:
columns_2 = pop_life_cl.columns

for i in columns_2:
    print(pop_life_cl[i].isnull().sum())
    print(i)

810
COU
0
Country
0
Year
810
Life Expectancy at Birth
0
Population


In [116]:
#Exporting to excell in order to visualize it quicker
pop_life_cl.to_csv(r'health_pop_view.csv', index = False)

Replacing NaN values with Mean by country

In [87]:
pop_life_fill = pop_life_cl.fillna(pop_life_cl.groupby('Country').transform('mean'))
pop_life_fill = pop_life_cl.fillna(pop_life_cl.groupby('Year').transform('mean'))

In [88]:
pop_life_fill.head()

,COU,Country,Year,Life Expectancy at Birth,Population
0,AUS,Australia,1964,70.7,11121600.0
31,AUS,Australia,1965,71.0,11340900.0
62,AUS,Australia,1966,70.8,11599498.0
93,AUS,Australia,1967,71.2,11799078.0
124,AUS,Australia,1968,70.9,12008635.0


In [14]:
#Checking out NaN Values
columns = pop_life_fill.columns

for i in columns:
    print(pop_life_fill[i].isnull().sum())
    print(i)

810
COU
0
Country
0
Year
0
Life Expectancy at Birth
0
Population


In [15]:
pop_life_fill.drop('COU', axis=1)

,Country,Year,Life Expectancy at Birth,Population
0,Australia,1964,70.700000,1.112160e+07
31,Australia,1965,71.000000,1.134090e+07
62,Australia,1966,70.800000,1.159950e+07
93,Australia,1967,71.200000,1.179908e+07
124,Australia,1968,70.900000,1.200864e+07
...,...,...,...,...
95325,OECD - Total,2014,78.972727,1.272146e+09
95356,OECD - Total,2015,78.956818,1.279864e+09
95387,OECD - Total,2016,79.213636,1.287731e+09
95418,OECD - Total,2017,79.340909,1.294830e+09


In [89]:
pop_life_fill.shape[0]

3080

In [51]:
pop_life_fill.to_csv(r'health_pop.csv', index = False)

Loading medals count info

In [154]:
file_to_load_trey = pd.read_csv('../Trey_Graham/Reviews_Edits/Use for analysis/08-03-21_medals_and_GDP.csv')

medal_count_df = pd.DataFrame(file_to_load_trey)

Replacing country names with standardized names

In [155]:
medal_count = medal_count_df['Country'].replace('Soviet Union', 'Russia', inplace=True)

In [156]:
medal_count = medal_count_df['Country'].replace('Russian Federation', 'Russia', inplace=True)

In [157]:
medal_count = medal_count_df['Country'].replace('Great Britain', 'England', inplace=True)
medal_count = medal_count_df['Country'].replace('United Kingdom', 'England', inplace=True)
medal_count = medal_count_df['Country'].replace('East Germany', 'Germany', inplace=True)
medal_count = medal_count_df['Country'].replace('West Germany', 'Germany', inplace=True)

In [158]:
medal_count = medal_count_df['Country'].replace("People's Republic of China", 'China', inplace=True)

Left Join (All from medal_count_df and only matching from stats)

In [159]:
medals_health_pop_stats = pd.merge(medal_count_df, pop_life_cl, how="left", on=["Country", "Year"])

In [160]:
medals_health_pop_stats.shape[0]

865

In [161]:
columns = medals_health_pop_stats.columns

for i in columns:
    print(medals_health_pop_stats[i].isnull().sum())
    print(i)

0
Country
0
Country Code
0
Gold
0
Silver
0
Bronze
0
Total
0
Year
0
Host_City
0
Host_Country
23
 GDP (USD Millions) 
450
COU
450
Life Expectancy at Birth
385
Population


In [162]:
dropped_medals = medals_health_pop_stats.dropna(axis=0)
dropped_medals.shape[0]

415

In [163]:
dropped_medals.head()

,Country,Country Code,Gold,Silver,Bronze,Total,Year,Host_City,Host_Country,GDP (USD Millions),COU,Life Expectancy at Birth,Population
0,Netherlands,NLD,2,4,4,10,1964,Tokyo,JPN,18699380731,NLD,73.8,12127116.0
3,Brazil,BRA,0,0,1,1,1964,Tokyo,JPN,21211892260,BRA,56.2,81168654.0
5,Sweden,SWE,2,2,4,8,1964,Tokyo,JPN,22532416750,SWE,73.8,7661351.0
8,Japan,JPN,16,5,8,29,1964,Tokyo,JPN,81749006382,JPN,70.3,98110000.0
11,Switzerland,CHE,1,2,1,4,1964,Tokyo,JPN,14480556572,CHE,72.2,5789228.0


In [164]:
# File to load into postgresDB
medals_health_pop_stats.to_csv(r'metal_count_stats.csv', index = False)

Right join verifies we only dropped non olympic years in the last file

In [166]:
medal_health_stats = pd.merge(medal_count_df, pop_life_cl, how="right", on=["Country", "Year"])

In [167]:
medal_health_stats['Host_City'] = medal_health_stats['Host_City'].fillna(0)

In [168]:
medal_health_stats.head()

,Country,Country Code,Gold,Silver,Bronze,Total,Year,Host_City,Host_Country,GDP (USD Millions),COU,Life Expectancy at Birth,Population
0,Australia,AUS,6.0,2.0,10.0,18.0,1964,Tokyo,JPN,23764139321,AUS,70.7,11121600.0
1,Australia,NaN,NaN,NaN,NaN,NaN,1965,0,NaN,NaN,AUS,71.0,11340900.0
2,Australia,NaN,NaN,NaN,NaN,NaN,1966,0,NaN,NaN,AUS,70.8,11599498.0
3,Australia,NaN,NaN,NaN,NaN,NaN,1967,0,NaN,NaN,AUS,71.2,11799078.0
4,Australia,AUS,5.0,7.0,5.0,17.0,1968,Mexico City,MEX,32665472057,AUS,70.9,12008635.0


In [169]:
medals_stats = medal_health_stats[medal_health_stats.Host_City != 0]

In [170]:
medals_stats.head()

,Country,Country Code,Gold,Silver,Bronze,Total,Year,Host_City,Host_Country,GDP (USD Millions),COU,Life Expectancy at Birth,Population
0,Australia,AUS,6.0,2.0,10.0,18.0,1964,Tokyo,JPN,23764139321,AUS,70.7,11121600.0
4,Australia,AUS,5.0,7.0,5.0,17.0,1968,Mexico City,MEX,32665472057,AUS,70.9,12008635.0
8,Australia,AUS,8.0,7.0,2.0,17.0,1972,Munich,DEU,51967289720,AUS,72.0,13303664.0
12,Australia,AUS,0.0,1.0,4.0,5.0,1976,Montreal,CAN,1.04921E+11,AUS,72.8,14033083.0
16,Australia,AUS,2.0,2.0,5.0,9.0,1980,Moscow,RUS,1.49776E+11,AUS,74.6,14695356.0


In [171]:
medals_stats.shape[0]

480

In [172]:
columns = medals_stats.columns

for i in columns:
    print(medals_stats[i].isnull().sum())
    print(i)

0
Country
0
Country Code
0
Gold
0
Silver
0
Bronze
0
Total
0
Year
0
Host_City
0
Host_Country
0
 GDP (USD Millions) 
65
COU
65
Life Expectancy at Birth
0
Population


Loading distance and GDP data

In [173]:
medals_health_pop_stats.columns

Index(['Country', 'Country Code', 'Gold', 'Silver', 'Bronze', 'Total', 'Year',
       'Host_City', 'Host_Country', ' GDP (USD Millions) ', 'COU',
       'Life Expectancy at Birth', 'Population'],
      dtype='object')

In [175]:
distance_csv = pd.read_csv("Country_Capitals_Distance.csv")
# gdp_csv = ('GDP_Per_Year.csv')

In [176]:
disctance_df = pd.DataFrame(distance_csv)

In [177]:
disctance_df.head()

,Country,Capital,GDP (Billions),Latitude,Longitude,Latitude.1,Longitude.1,Latitude Hemisphere,Longitude Hemisphere,Distance From Tokyo 1964,...,Distance From Los Angeles 1984,Distance From Seoul 1988,Distance From Barcelona 1992,Distance From Atlanta 1996,Distance From Sydney 2000,Distance From Athens 2004,Distance From Beijing 2008,Distance From London 2012,Distance From Rio de Janeiro 2016,Distance From Tokyo 2020
0,Afghanistan,Kabul,19.29,34.28N,69.11E,34.28,69.11,North,East,3852.695234,...,10229.322310,3140.460940,3687.702265,8381.179784,6496.640378,2502.869092,2596.700595,3955.769518,7290.724741,3852.695234
1,Albania,Tirane,15.29,41.18N,19.49E,41.18,19.49,North,East,6571.702756,...,7536.133412,5848.844830,945.783004,5694.970611,8862.276379,329.679124,5281.563651,1277.354816,5591.135578,6571.702756
2,Algeria,Algiers,171.00,36.42N,03.08E,36.42,3.08,North,East,7456.896815,...,6626.090244,6739.011809,346.510726,4779.414037,9430.625168,1115.622926,6180.126784,1044.929886,4809.973012,7456.896815
3,American Samoa,Pago Pago,NaN,14.16S,170.43W,-14.16,-170.40,South,West,17274.499860,...,4379.072411,16595.202510,10172.101930,5742.998762,17612.491920,11170.603740,16081.216760,10341.615120,6972.944914,17274.499860
4,Andorra,Andorra la Vella,3.22,42.31N,01.32E,42.31,1.32,North,East,7566.659669,...,6552.808731,6843.529703,78.635860,4716.891500,9726.649424,1252.126668,6275.338342,628.914256,5113.942627,7566.659669


In [178]:
disctance_df.shape[0]

200

In [179]:
# file_G = ("GDP_Per_Year.csv")
# gdp_df = pd.read_csv(file_G)
# gdp_df.head()

In [180]:
#getting number of columns in gdp data
# gdp_df.shape[0]

In [181]:
# Standardizing column names
# gdp_df = gdp_df.rename(columns={"TIME":"Year"})
# gdp_df_drop = gdp_df.drop(["LOCATION", "INDICATOR", "SUBJECT", "FREQUENCY", "Flag Codes", "MEASURE"], axis=1)

In [182]:
# gdp_distance = pd.merge(disctance_df, gdp_df_drop, how="right", on="Country")

In [184]:
# gdp_distance.head()

In [185]:
# gdp_distance.shape[0]

In [186]:
disctance_df = disctance_df.drop(["Latitude", "Longitude", "Latitude.1", "Longitude.1"], axis=1)

In [187]:
disctance_df.head()

,Country,Capital,GDP (Billions),Latitude Hemisphere,Longitude Hemisphere,Distance From Tokyo 1964,Distance From Mexico City 1968,Distance From Munich 1972,Distance From Montreal 1976,Distance From Moscow 1980,Distance From Los Angeles 1984,Distance From Seoul 1988,Distance From Barcelona 1992,Distance From Atlanta 1996,Distance From Sydney 2000,Distance From Athens 2004,Distance From Beijing 2008,Distance From London 2012,Distance From Rio de Janeiro 2016,Distance From Tokyo 2020
0,Afghanistan,Kabul,19.29,North,East,3852.695234,9243.020276,3283.494195,7937.373451,2267.337882,10229.322310,3140.460940,3687.702265,8381.179784,6496.640378,2502.869092,2596.700595,3955.769518,7290.724741,3852.695234
1,Albania,Tirane,15.29,North,East,6571.702756,6650.258831,645.876130,5198.828391,1385.794163,7536.133412,5848.844830,945.783004,5694.970611,8862.276379,329.679124,5281.563651,1277.354816,5591.135578,6571.702756
2,Algeria,Algiers,171.00,North,East,7456.896815,5704.149817,932.390558,4339.752748,2285.897025,6626.090244,6739.011809,346.510726,4779.414037,9430.625168,1115.622926,6180.126784,1044.929886,4809.973012,7456.896815
3,American Samoa,Pago Pago,NaN,South,West,17274.499860,4522.583406,10826.128760,6614.730929,12318.146190,4379.072411,16595.202510,10172.101930,5742.998762,17612.491920,11170.603740,16081.216760,10341.615120,6972.944914,17274.499860
4,Andorra,Andorra la Vella,3.22,North,East,7566.659669,5710.101626,689.667102,4203.960189,2166.115810,6552.808731,6843.529703,78.635860,4716.891500,9726.649424,1252.126668,6275.338342,628.914256,5113.942627,7566.659669


In [191]:
disctance_df.dropna(axis=0)

,Country,Capital,GDP (Billions),Latitude Hemisphere,Longitude Hemisphere,Distance From Tokyo 1964,Distance From Mexico City 1968,Distance From Munich 1972,Distance From Montreal 1976,Distance From Moscow 1980,Distance From Los Angeles 1984,Distance From Seoul 1988,Distance From Barcelona 1992,Distance From Atlanta 1996,Distance From Sydney 2000,Distance From Athens 2004,Distance From Beijing 2008,Distance From London 2012,Distance From Rio de Janeiro 2016,Distance From Tokyo 2020
0,Afghanistan,Kabul,19.29,North,East,3852.695234,9243.020276,3283.494195,7937.373451,2267.337882,10229.322310,3140.460940,3687.702265,8381.179784,6496.640378,2502.869092,2596.700595,3955.769518,7290.724741,3852.695234
1,Albania,Tirane,15.29,North,East,6571.702756,6650.258831,645.876130,5198.828391,1385.794163,7536.133412,5848.844830,945.783004,5694.970611,8862.276379,329.679124,5281.563651,1277.354816,5591.135578,6571.702756
2,Algeria,Algiers,171.00,North,East,7456.896815,5704.149817,932.390558,4339.752748,2285.897025,6626.090244,6739.011809,346.510726,4779.414037,9430.625168,1115.622926,6180.126784,1044.929886,4809.973012,7456.896815
4,Andorra,Andorra la Vella,3.22,North,East,7566.659669,5710.101626,689.667102,4203.960189,2166.115810,6552.808731,6843.529703,78.635860,4716.891500,9726.649424,1252.126668,6275.338342,628.914256,5113.942627,7566.659669
5,Angola,Luanda,89.42,South,East,7549.710730,6419.958724,3909.100005,6111.341675,4606.135478,7751.422936,6949.376417,3494.223297,6071.374855,7738.665587,3228.967809,6530.700463,4192.749198,3231.823078,7549.710730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Uzbekistan,Tashkent,57.73,North,East,3870.814474,9308.331593,3176.889203,7904.552329,1992.981727,10240.654630,3145.512780,3654.401516,8396.304708,6850.530992,2504.431063,2574.178894,3840.121994,7559.387370,3870.814474
194,Vanuatu,Port-Vila,0.93,South,East,3981.493991,14806.670950,9670.368673,13995.463800,8733.070085,16032.917580,4408.592188,9927.882584,14231.416080,1460.850935,8762.749730,4847.532505,10336.006810,11541.492450,3981.493991
195,Venezuela,Caracas,371.00,North,West,11393.903810,1880.336481,5001.499562,2478.228765,6472.067202,3263.545939,10704.023220,4322.067839,1888.620258,12274.099930,5262.688568,10180.216620,4605.444286,2623.070951,11393.903810
198,Zambia,Lusaka,23.31,South,East,7029.923361,7344.414132,4468.636382,7047.350434,4906.097072,8687.964851,6480.555997,4159.761160,7014.913711,6839.882331,3656.356506,6117.207457,4849.277754,3930.040716,7029.923361


In [192]:
disctance_df.shape[0]

200

In [195]:
disctance_df = disctance_df.drop('GDP (Billions)', axis=1)

In [197]:
disctance_df.columns

Index(['Country', 'Capital', 'Latitude Hemisphere', 'Longitude Hemisphere',
       'Distance From Tokyo 1964', 'Distance From Mexico City 1968',
       'Distance From Munich 1972', 'Distance From Montreal 1976',
       'Distance From Moscow 1980', 'Distance From Los Angeles 1984',
       'Distance From Seoul 1988', 'Distance From Barcelona 1992',
       'Distance From Atlanta 1996', 'Distance From Sydney 2000',
       'Distance From Athens 2004', 'Distance From Beijing 2008',
       'Distance From London 2012', 'Distance From Rio de Janeiro 2016',
       'Distance From Tokyo 2020'],
      dtype='object')

In [198]:
# File to load into postgresDB
disctance_df.to_csv(r'distance.csv', index = False)

In [4]:
import pandas as pd

In [50]:
file = 'C:/Users/tucke/OneDrive/Desktop/Data Analysis Bootcamp/Olympics/HH/Aug2/Competition_Cleaned.csv'

masterReseultData = pd.read_csv(file)

masterReseultData_df = pd.DataFrame(masterReseultData)

In [87]:
file_2 = 'C:/Users/tucke/OneDrive/Desktop/Data Analysis Bootcamp/Olympics/Trey_Graham/Reviews_Edits/Use for analysis/country_codes_table (version 1).xlsb.csv'

noc = pd.read_csv(file_2)

noc_df = pd.DataFrame(noc)

In [88]:
noc = noc.rename(columns={'ISO 3166 ALPHA-3':'NOC'})
noc.head()

,NOC,NOC.1,Country
0,AFG,AFG,Afghanistan
1,AHO,AHO,Netherlands Antilles
2,ALB,ALB,Albania
3,ALG,ALG,Algeria
4,AND,AND,Andorra


In [89]:
masterReseultData_df.head()

,Event,NOC,Pos.,Year,Discipline,Gender Cleaned,Event Cleaned
0,"Balance Beam, Women",TCH,1,1964,Artistic Gymnastics,Women,Balance Beam
1,"Balance Beam, Women",URS,2,1964,Artistic Gymnastics,Women,Balance Beam
2,"Balance Beam, Women",URS,3,1964,Artistic Gymnastics,Women,Balance Beam
3,"Floor Exercise, Men",ITA,1,1964,Artistic Gymnastics,Men,Floor Exercise
4,"Floor Exercise, Men",URS,2,1964,Artistic Gymnastics,Men,Floor Exercise


In [90]:
master_df = masterReseultData_df.drop('Event', axis=1)
master_df = master_df.rename(columns={'Pos.':'Place'})
master_df.head()

,NOC,Place,Year,Discipline,Gender Cleaned,Event Cleaned
0,TCH,1,1964,Artistic Gymnastics,Women,Balance Beam
1,URS,2,1964,Artistic Gymnastics,Women,Balance Beam
2,URS,3,1964,Artistic Gymnastics,Women,Balance Beam
3,ITA,1,1964,Artistic Gymnastics,Men,Floor Exercise
4,URS,2,1964,Artistic Gymnastics,Men,Floor Exercise


In [91]:
master_df.Place.unique()

array([1, 2, 3], dtype=int64)

In [92]:
master_df['Place'] = master_df['Place'].map({1:'Gold', 2:'Silver', 3:'Bronze'}, na_action=None)
master_df.head()

,NOC,Place,Year,Discipline,Gender Cleaned,Event Cleaned
0,TCH,Gold,1964,Artistic Gymnastics,Women,Balance Beam
1,URS,Silver,1964,Artistic Gymnastics,Women,Balance Beam
2,URS,Bronze,1964,Artistic Gymnastics,Women,Balance Beam
3,ITA,Gold,1964,Artistic Gymnastics,Men,Floor Exercise
4,URS,Silver,1964,Artistic Gymnastics,Men,Floor Exercise


In [101]:
master = pd.merge(noc, master_df, how='right', on=['NOC'])
master = master.dropna(axis=0)
master = master.drop('NOC.1', axis=1)
master.head()

,NOC,Country,Place,Year,Discipline,Gender Cleaned,Event Cleaned
0,TCH,Czechoslovakia,Gold,1964,Artistic Gymnastics,Women,Balance Beam
1,TCH,Czechoslovakia,Gold,1964,Artistic Gymnastics,Women,Horse Vault
2,TCH,Czechoslovakia,Gold,1964,Artistic Gymnastics,Women,Individual All-Around
3,TCH,Czechoslovakia,Silver,1964,Artistic Gymnastics,Women,Team All-Around
4,TCH,Czechoslovakia,Silver,1964,Athletics,Men,"1,500 metres"


In [97]:
master['Country'].isnull().sum()

0

In [98]:
master.shape[0]

10520

In [19]:
master['Event Cleaned'].columns

NameError: name 'master' is not defined

In [102]:
master.to_csv("events.csv", index=None)

In [12]:
import pandas as pd
file_cnt = 'eventCountryCount.csv'

eventCountryCount = pd.read_csv(file_cnt)

In [13]:
eventCountryCount = eventCountryCount.fillna(0)

In [24]:
eventCountryCount.head()

,Row Labels,"1,000 metres Time Trial","1,500 metres","1,500 metres Freestyle",10 kilometres Open Water,10 kilometres Walk,"10,000 metres",100 kilometres Team Time Trial,100 metres,100 metres Backstroke,...,Two Person Heavyweight Dinghy,Two Person Keelboat,Uneven Bars,Volleyball,Water Polo,Welterweight,"Welterweight, Freestyle","Welterweight, Greco-Roman",Windsurfer,Grand Total
0,Grand Total,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,United States,0.060606,0.054795,0.285714,0.055556,0.0,0.064516,0.043478,0.363636,0.433735,...,0.090909,0.175,0.186047,0.120482,0.147541,0.065217,0.152174,0.000000,0.043478,0.130703
2,Russia,0.060606,0.123288,0.071429,0.055556,0.2,0.032258,0.130435,0.038961,0.048193,...,0.000000,0.075,0.232558,0.216867,0.147541,0.065217,0.130435,0.113636,0.021739,0.109886
3,Germany,0.151515,0.082192,0.047619,0.111111,0.0,0.000000,0.130435,0.064935,0.132530,...,0.000000,0.000,0.162791,0.024096,0.000000,0.054348,0.000000,0.022727,0.021739,0.079373
4,China,0.000000,0.013699,0.023810,0.000000,0.6,0.032258,0.000000,0.000000,0.024096,...,0.000000,0.000,0.186047,0.072289,0.000000,0.032609,0.000000,0.000000,0.086957,0.051806


In [ ]:
for column in eventCountryCount.columns:
    